# SVD between a Model and Satellite data

This notebook shows how to multiply two matrices and calculate SVD. Each matrix is created out a set of GeoTiffs for a series of years. Both matrices should have the same dimension.

For demonstration we will use from a model (spring-index) and from a satellite (AVHRR).

## Dependencies

In [1]:

import java.io.{ByteArrayInputStream, ByteArrayOutputStream, ObjectInputStream, ObjectOutputStream}

import geotrellis.proj4.CRS
import geotrellis.raster.io.geotiff.writer.GeoTiffWriter
import geotrellis.raster.io.geotiff.{SinglebandGeoTiff, _}
import geotrellis.raster.{CellType, DoubleArrayTile, Tile, UByteCellType}
import geotrellis.spark.io.hadoop._
import geotrellis.vector.{Extent, ProjectedExtent}
import org.apache.hadoop.io.SequenceFile.Writer
import org.apache.hadoop.io.{SequenceFile, _}
import org.apache.spark.broadcast.Broadcast
import org.apache.spark.mllib.linalg._
import org.apache.spark.mllib.linalg.distributed._
import org.apache.spark.mllib.stat.{MultivariateStatisticalSummary, Statistics}
import org.apache.spark.rdd.RDD
import org.apache.spark.storage.StorageLevel
import org.apache.spark.{SparkConf, SparkContext}

import scala.sys.process._

## Mode of operation

Here the user can define the mode of operation.
* **rdd_offline_mode**: If false it means the notebook will create all data from scratch and store protected_extent and num_cols_rows into HDFS. Otherwise, these data structures are read from HDFS.

It is also possible to define which directory of GeoTiffs is to be used and on which **band** to run Kmeans. The options are
* **all** which are a multi-band (**8 bands**) GeoTiffs
* Or choose single band ones:
    0. Onset_Greenness_Increase
    1. Onset_Greenness_Maximum
    2. Onset_Greenness_Decrease
    3. Onset_Greenness_Minimum
    4. NBAR_EVI_Onset_Greenness_Minimum
    5. NBAR_EVI_Onset_Greenness_Maximum
    6. NBAR_EVI_Area
    7. Dynamics_QC

<span style="color:red">Note that when using a range **kemans offline mode** is not possible and it will be reset to **online mode**</span>.

### Mode of Operation setup
<a id='mode_of_operation_setup'></a>

In [2]:
var model_rdd_offline_mode = true
var model_matrix_offline_mode = true
var satellite_rdd_offline_mode = true
var satellite_matrix_offline_mode = true

//Using spring-index model
var model_path = "hdfs:///user/hadoop/spring-index/"
var model_dir = "BloomGridmet"
var model_band_num = 3  //First band is 0

//Using AVHRR Satellite data
//var satellite_path = "hdfs:///user/hadoop/avhrr/"
//var satellite_dir = "SOSTLowPR"
var satellite_path = "hdfs:///user/hadoop/spring-index/"
var satellite_dir = "LeafGridmet"
var sat_band_num = 3  //First band is 0

var out_path = "hdfs:///user/pheno/svd/spark/" + model_dir + satellite_dir + "/"

//Years between (inclusive) 1989 - 2014
var satellite_first_year = 1989
var satellite_last_year = 2014

//Years between (inclusive) 1980 - 2015
var model_first_year = 1989
var model_last_year = 2014

//Mask
val toBeMasked = true
//val mask_path = "hdfs:///user/hadoop/usa_mask_low.tif"
val mask_path = "hdfs:///user/hadoop/usa_mask_gridmet.tif"

//Matrix Mode: 0 Normal, 1 SC, 2 SR
val matrix_mode = 0

val save_rdds = true
val save_matrix = true

model_rdd_offline_mode = true
model_matrix_offline_mode = true
satellite_rdd_offline_mode = true
satellite_matrix_offline_mode = true
model_path = hdfs:///user/hadoop/spring-index/
model_dir = BloomGridmet
model_band_num = 3
satellite_path = hdfs:///user/hadoop/spring-index/
satellite_dir = LeafGridmet
sat_band_num = 3
out_path = hdfs:///user/pheno/svd/spark/BloomGridmetLeafGridmet/
satellite_first_year = 1989
satellite_last_year = 2014
model_first_year = 1989
model_last_year = 2014
toBeMasked = true
mask_path = hdfs:///user/hadoop/usa_mask_gridmet.tif
matrix_mode = 0
save_rdds = true
save_matrix = true


true


<span style="color:red">DON'T MODIFY ANY PIECE OF CODE FROM HERE ON!!!</span>.


### Mode of operation validation

In [3]:
//Check offline modes
var conf = sc.hadoopConfiguration
var fs = org.apache.hadoop.fs.FileSystem.get(conf)

//Paths to store data structures for Offline runs
var mask_str = ""
if (toBeMasked)
  mask_str = "_mask"
var model_grid0_path = out_path + model_dir + "_grid0"
var model_grid0_index_path = out_path + model_dir + "_grid0_index"

var model_grid_path = out_path + model_dir + "_grid"
var satellite_grid_path = out_path + satellite_dir + "_grid"
var model_matrix_path = out_path + model_dir + "_matrix"
var satellite_matrix_path = out_path + satellite_dir + "_matrix"
var metadata_path = out_path + model_dir + "_metadata"

var sc_path = out_path + model_dir + "_sc"
var mc_path = out_path + model_dir + "_mc"

val model_rdd_offline_exists = fs.exists(new org.apache.hadoop.fs.Path(model_grid_path))
val model_matrix_offline_exists = fs.exists(new org.apache.hadoop.fs.Path(model_matrix_path))
val satellite_rdd_offline_exists = fs.exists(new org.apache.hadoop.fs.Path(satellite_grid_path))
val satellite_matrix_offline_exists = fs.exists(new org.apache.hadoop.fs.Path(satellite_matrix_path))

if (model_rdd_offline_mode != model_rdd_offline_exists) {
  println("\"Load GeoTiffs\" offline mode is not set properly, i.e., either it was set to false and the required file does not exist or vice-versa. We will reset it to " + model_rdd_offline_exists.toString())
  model_rdd_offline_mode = model_rdd_offline_exists
}

if (model_matrix_offline_mode != model_matrix_offline_exists) {
  println("\"Matrix\" offline mode is not set properly, i.e., either it was set to false and the required file does not exist or vice-versa. We will reset it to " + model_matrix_offline_exists.toString())
  model_matrix_offline_mode = model_matrix_offline_exists
}

var model_skip_rdd = false
if (model_matrix_offline_exists) {
    println("Since we have a matrix, the load of the grids RDD will be skipped!!!")
    model_skip_rdd = true
}

if (satellite_rdd_offline_mode != satellite_rdd_offline_exists) {
  println("\"Load GeoTiffs\" offline mode is not set properly, i.e., either it was set to false and the required file does not exist or vice-versa. We will reset it to " + satellite_rdd_offline_exists.toString())
  satellite_rdd_offline_mode = satellite_rdd_offline_exists
}

if (satellite_matrix_offline_mode != satellite_matrix_offline_exists) {
  println("\"Matrix\" offline mode is not set properly, i.e., either it was set to false and the required file does not exist or vice-versa. We will reset it to " + satellite_matrix_offline_exists.toString())
  satellite_matrix_offline_mode = satellite_matrix_offline_exists
}

var satellite_skip_rdd = false
if (satellite_matrix_offline_exists) {
    println("Since we have a matrix, the load of the grids RDD will be skipped!!!")
    satellite_skip_rdd = true
}

var corr_tif = out_path + "_" + satellite_dir + "_" + model_dir + ".tif"
var corr_tif_tmp = "/tmp/svd_" + satellite_dir + "_" + model_dir + ".tif"

//Years
val model_years = 1989 to 2014
val satellite_years = 1989 to 2014

if (!satellite_years.contains(satellite_first_year) || !(satellite_years.contains(satellite_last_year))) {
  println("Invalid range of years for " + satellite_dir + ". I should be between " + satellite_first_year + " and " + satellite_last_year)
  System.exit(0)
}

if (!model_years.contains(model_first_year) || !(model_years.contains(model_last_year))) {
  println("Invalid range of years for " + model_dir + ". I should be between " + model_first_year + " and " + model_last_year)
  System.exit(0)
}

if ( ((satellite_last_year - model_first_year) > (model_last_year - model_first_year)) || ((satellite_last_year - model_first_year) > (model_last_year - model_first_year))) {
  println("The range of years for each data set should be of the same length.");
  System.exit(0)
}

var model_years_range = (model_years.indexOf(model_first_year), model_years.indexOf(model_last_year))
var satellite_years_range = (satellite_years.indexOf(satellite_first_year), satellite_years.indexOf(satellite_last_year))

//Global variables
var projected_extent = new ProjectedExtent(new Extent(0,0,0,0), CRS.fromName("EPSG:3857"))
var model_grid0: RDD[(Long, Double)] = sc.emptyRDD
var model_grid0_index: RDD[Long] = sc.emptyRDD
var grids_RDD: RDD[Array[Double]] = sc.emptyRDD
var model_grids_RDD: RDD[Array[Double]] = sc.emptyRDD
var model_grids: RDD[Array[Double]] = sc.emptyRDD
val rows = sc.parallelize(Array[Double]()).map(m => Vectors.dense(m))
var model_summary: MultivariateStatisticalSummary = new RowMatrix(rows).computeColumnSummaryStatistics()
var model_std :Array[Double] = new Array[Double](0)

var satellite_grids_RDD: RDD[Array[Double]] = sc.emptyRDD
var satellite_grids: RDD[Array[Double]] = sc.emptyRDD
var satellite_summary: MultivariateStatisticalSummary = new RowMatrix(rows).computeColumnSummaryStatistics()
var satellite_std :Array[Double] = new Array[Double](0)

var num_cols_rows :(Int, Int) = (0, 0)
var cellT :CellType = UByteCellType
var mask_tile0 :Tile = new SinglebandGeoTiff(geotrellis.raster.ArrayTile.empty(cellT, num_cols_rows._1, num_cols_rows._2), projected_extent.extent, projected_extent.crs, Tags.empty, GeoTiffOptions.DEFAULT).tile
var satellite_cells_size :Long = 0
var model_cells_size :Long = 0
var t0 : Long = 0
var t1 : Long = 0

Waiting for a Spark session to start...

"Matrix" offline mode is not set properly, i.e., either it was set to false and the required file does not exist or vice-versa. We will reset it to false
"Matrix" offline mode is not set properly, i.e., either it was set to false and the required file does not exist or vice-versa. We will reset it to false
[Stage 2:=====================================================>(994 + 6) / 1000]

conf = Configuration: core-default.xml, core-site.xml, mapred-default.xml, mapred-site.xml, yarn-default.xml, yarn-site.xml, hdfs-default.xml, hdfs-site.xml, file:/usr/lib/spark-2.1.1-bin-without-hadoop/conf/hive-site.xml
fs = DFS[DFSClient[clientName=DFSClient_NONMAPREDUCE_-1473718442_41, ugi=pheno (auth:SIMPLE)]]
mask_str = _mask
model_grid0_path = hdfs:///user/pheno/svd/spark/BloomGridmetLeafGridmet/BloomGridmet_grid0
model_grid0_index_path = hdfs:///user/pheno/svd/spark/BloomGridmetLeafGridmet/BloomGridmet_grid0_index
model_grid_path = hdfs:///user/pheno/svd/spark/BloomGridmetLeafGridmet/BloomGridmet_grid
satellite_grid_path = hdfs:///user/pheno/svd/spark/BloomGridmetLeafGr...


hdfs:///user/pheno/svd/spark/BloomGridmetLeafGridmet/LeafGridmet_grid

## Functions to (de)serialize any structure into Array[Byte]

In [4]:
def serialize(value: Any): Array[Byte] = {
    val out_stream: ByteArrayOutputStream = new ByteArrayOutputStream()
    val obj_out_stream = new ObjectOutputStream(out_stream)
    obj_out_stream.writeObject(value)
    obj_out_stream.close
    out_stream.toByteArray
}

def deserialize(bytes: Array[Byte]): Any = {
    val obj_in_stream = new ObjectInputStream(new ByteArrayInputStream(bytes))
    val value = obj_in_stream.readObject
    obj_in_stream.close
    value
}

serialize: (value: Any)Array[Byte]
deserialize: (bytes: Array[Byte])Any


## Load GeoTiffs

Using GeoTrellis all GeoTiffs of a directory will be loaded into a RDD. Using the RDD, we extract a grid from the first file to lated store the Kmeans cluster_IDS, we build an Index for populate such grid and we filter out here all NaN values.

In [5]:
t0 = System.nanoTime()

//Load Mask
if (toBeMasked) {
  val mask_tiles_RDD = sc.hadoopGeoTiffRDD(mask_path).values
  val mask_tiles_withIndex = mask_tiles_RDD.zipWithIndex().map{case (e,v) => (v,e)}
  mask_tile0 = (mask_tiles_withIndex.filter(m => m._1==0).filter(m => !m._1.isNaN).values.collect())(0)
}

//Local variables
val pattern: String = "tif"
val satellite_filepath: String = satellite_path + satellite_dir
val model_filepath: String = model_path + "/" + model_dir

t1 = System.nanoTime()
println("Elapsed time: " + (t1 - t0) + "ns")

Elapsed time: 1702500743ns


t0 = 1158682843545
pattern = tif
satellite_filepath = hdfs:///user/hadoop/spring-index/LeafGridmet
model_filepath = hdfs:///user/hadoop/spring-index//BloomGridmet
t1 = 1160385344288


1160385344288

### Satellite data

In [6]:
t0 = System.nanoTime()

if (satellite_rdd_offline_mode) {
  satellite_grids_RDD = sc.objectFile(satellite_grid_path)
} else {
  //Lets load MODIS Singleband GeoTiffs and return RDD just with the tiles.
  if (sat_band_num == 0) {
    val satellite_geos_RDD = sc.hadoopGeoTiffRDD(satellite_filepath, pattern)
    val satellite_tiles_RDD = satellite_geos_RDD.values

    if (toBeMasked) {
      val mask_tile_broad: Broadcast[Tile] = sc.broadcast(mask_tile0)
      satellite_grids_RDD = satellite_tiles_RDD.map(m => m.localInverseMask(mask_tile_broad.value, 1, -1000).toArrayDouble().filter(_ != -1000))
    } else {
      satellite_grids_RDD = satellite_tiles_RDD.map(m => m.toArrayDouble())
    }
  } else {
    val satellite_geos_RDD = sc.hadoopMultibandGeoTiffRDD(satellite_filepath, pattern)
    val satellite_tiles_RDD = satellite_geos_RDD.values

    val band_numB: Broadcast[Int] = sc.broadcast(sat_band_num)
    if (toBeMasked) {
      val mask_tile_broad: Broadcast[Tile] = sc.broadcast(mask_tile0)
      satellite_grids_RDD = satellite_tiles_RDD.map(m => m.band(band_numB.value).localInverseMask(mask_tile_broad.value, 1, -1000).toArrayDouble().filter(_ != -1000))
    } else {
      satellite_grids_RDD = satellite_tiles_RDD.map(m => m.band(band_numB.value).toArrayDouble())
    }
  }

  //Store in HDFS
  if (save_rdds) {
      satellite_grids_RDD.saveAsObjectFile(satellite_grid_path)
  }
}
val satellite_grids_withIndex = satellite_grids_RDD.zipWithIndex().map { case (e, v) => (v, e) }

//Filter out the range of years:
satellite_grids = satellite_grids_withIndex.filterByRange(satellite_years_range._1, satellite_years_range._2).values
//satellite_grids.persist(StorageLevel.DISK_ONLY)

//Collect Stats:
satellite_summary = Statistics.colStats(satellite_grids.map(m => Vectors.dense(m)))
satellite_std = satellite_summary.variance.toArray.map(m => scala.math.sqrt(m))

var satellite_grid0_index: RDD[Double] = satellite_grids_withIndex.filter(m => m._1 == 0).values.flatMap(m => m)
satellite_cells_size = satellite_grid0_index.count().toInt
println("Number of cells is: " + satellite_cells_size)

t1 = System.nanoTime()
println("Elapsed time: " + (t1 - t0) + "ns")

[Stage 7:===============================================>           (4 + 1) / 5]Number of cells is: 483850
Elapsed time: 3001483778ns


t0 = 1163045238075
satellite_grids_withIndex = MapPartitionsRDD[28] at map at <console>:120
satellite_grids = MapPartitionsRDD[30] at values at <console>:123
satellite_summary = org.apache.spark.mllib.stat.MultivariateOnlineSummarizer@761d8c65
satellite_std = [D@332a42d0
satellite_grid0_index = MapPartitionsRDD[38] at flatMap at <console>:130
satellite_cells_size = 483850
t1 = 1166046721853


1166046721853

### Model data

In [7]:
t0 = System.nanoTime()

if (model_rdd_offline_mode) {
  model_grids_RDD = sc.objectFile(model_grid_path)
  model_grid0 = sc.objectFile(model_grid0_path)
  model_grid0_index = sc.objectFile(model_grid0_index_path)
  val metadata = sc.sequenceFile(metadata_path, classOf[IntWritable], classOf[BytesWritable]).map(_._2.copyBytes()).collect()
  projected_extent = deserialize(metadata(0)).asInstanceOf[ProjectedExtent]
  num_cols_rows = (deserialize(metadata(1)).asInstanceOf[Int], deserialize(metadata(2)).asInstanceOf[Int])
  cellT = deserialize(metadata(3)).asInstanceOf[CellType]
} else {
  if (model_band_num != 0) {
    val model_geos_RDD = sc.hadoopMultibandGeoTiffRDD(model_filepath, pattern)
    val model_tiles_RDD = model_geos_RDD.values

    //Retrieve the number of cols and rows of the Tile's grid
    val tiles_withIndex = model_tiles_RDD.zipWithIndex().map { case (v, i) => (i, v) }
    val tile0 = (tiles_withIndex.filter(m => m._1 == 0).values.collect()) (0)

    num_cols_rows = (tile0.cols, tile0.rows)
    cellT = tile0.cellType

    //Retrieve the ProjectExtent which contains metadata such as CRS and bounding box
    val projected_extents_withIndex = model_geos_RDD.keys.zipWithIndex().map { case (e, v) => (v, e) }
    projected_extent = (projected_extents_withIndex.filter(m => m._1 == 0).values.collect()) (0)

    val band_numB: Broadcast[Int] = sc.broadcast(model_band_num)
    if (toBeMasked) {
      val mask_tile_broad: Broadcast[Tile] = sc.broadcast(mask_tile0)
      grids_RDD = model_tiles_RDD.map(m => m.band(band_numB.value).localInverseMask(mask_tile_broad.value, 1, -1000).toArrayDouble())
    } else {
      grids_RDD = model_tiles_RDD.map(m => m.band(band_numB.value).toArrayDouble())
    }
  } else {
    val model_geos_RDD = sc.hadoopGeoTiffRDD(model_filepath, pattern)
    val model_tiles_RDD = model_geos_RDD.values

    //Retrieve the number of cols and rows of the Tile's grid
    val tiles_withIndex = model_tiles_RDD.zipWithIndex().map { case (v, i) => (i, v) }
    val tile0 = (tiles_withIndex.filter(m => m._1 == 0).values.collect()) (0)

    num_cols_rows = (tile0.cols, tile0.rows)
    cellT = tile0.cellType

    //Retrieve the ProjectExtent which contains metadata such as CRS and bounding box
    val projected_extents_withIndex = model_geos_RDD.keys.zipWithIndex().map { case (e, v) => (v, e) }
    projected_extent = (projected_extents_withIndex.filter(m => m._1 == 0).values.collect()) (0)

    if (toBeMasked) {
      val mask_tile_broad: Broadcast[Tile] = sc.broadcast(mask_tile0)
      grids_RDD = model_tiles_RDD.map(m => m.localInverseMask(mask_tile_broad.value, 1, -1000).toArrayDouble())
    } else {
      grids_RDD = model_tiles_RDD.map(m => m.toArrayDouble())
    }
  }

  //Get Index for each Cell
  val grids_withIndex = grids_RDD.zipWithIndex().map { case (e, v) => (v, e) }
  if (toBeMasked) {
    model_grid0_index = grids_withIndex.filter(m => m._1 == 0).values.flatMap(m => m).zipWithIndex.filter(m => m._1 != -1000.0).map { case (v, i) => (i) }
  } else {
    model_grid0_index = grids_withIndex.filter(m => m._1 == 0).values.flatMap(m => m).zipWithIndex.map { case (v, i) => (i) }
  }

  //Get the Tile's grid
  model_grid0 = grids_withIndex.filter(m => m._1 == 0).values.flatMap( m => m).zipWithIndex.map{case (v,i) => (i,v)}

  //Lets filter out NaN
  if (toBeMasked) {
    model_grids_RDD = grids_RDD.map(m => m.filter(m => m != -1000.0))
  } else {
    model_grids_RDD = grids_RDD
  }

  //Store data in HDFS
  model_grids_RDD.saveAsObjectFile(model_grid_path)
  model_grid0.saveAsObjectFile(model_grid0_path)
  model_grid0_index.saveAsObjectFile(model_grid0_index_path)

  val writer: SequenceFile.Writer = SequenceFile.createWriter(conf,
    Writer.file(metadata_path),
    Writer.keyClass(classOf[IntWritable]),
    Writer.valueClass(classOf[BytesWritable])
  )

  writer.append(new IntWritable(1), new BytesWritable(serialize(projected_extent)))
  writer.append(new IntWritable(2), new BytesWritable(serialize(num_cols_rows._1)))
  writer.append(new IntWritable(3), new BytesWritable(serialize(num_cols_rows._2)))
  writer.append(new IntWritable(4), new BytesWritable(serialize(cellT)))
  writer.hflush()
  writer.close()
}

val model_grids_withIndex = model_grids_RDD.zipWithIndex().map { case (e, v) => (v, e) }

//Filter out the range of years:
model_grids = model_grids_withIndex.filterByRange(model_years_range._1, model_years_range._2).values
//model_grids.persist(StorageLevel.DISK_ONLY)

//Collect Stats:
model_summary = Statistics.colStats(model_grids.map(m => Vectors.dense(m)))
//model_std = model_summary.variance.toArray.map(m => scala.math.sqrt(m))

var model_tile0_index: RDD[Double] = model_grids_withIndex.filter(m => m._1 == 0).values.flatMap(m => m)
model_cells_size = model_tile0_index.count().toInt

t1 = System.nanoTime()
println("Elapsed time: " + (t1 - t0) + "ns")

[Stage 12:==============================================>           (4 + 1) / 5]Elapsed time: 2432090908ns


t0 = 1169174905705
model_grids_withIndex = MapPartitionsRDD[48] at map at <console>:187
model_grids = MapPartitionsRDD[50] at values at <console>:190
model_summary = org.apache.spark.mllib.stat.MultivariateOnlineSummarizer@772abde6
model_tile0_index = MapPartitionsRDD[58] at flatMap at <console>:197
model_cells_size = 483850
t1 = 1171606996613


1171606996613

## Matrices

### Satellite

In [8]:
//Satellite
val satellite_cells_sizeB = sc.broadcast(satellite_cells_size)
val satellite_mat: RowMatrix = new RowMatrix(satellite_grids.map(m => m.zipWithIndex).map(m => m.filter(!_._1.isNaN)).map(m => Vectors.sparse(satellite_cells_sizeB.value.toInt, m.map(v => v._2), m.map(v => v._1))))

// Split the matrix into one number per line.
val sat_byColumnAndRow = satellite_mat.rows.zipWithIndex.map {
  case (row, rowIndex) => row.toArray.zipWithIndex.map {
    case (number, columnIndex) => new MatrixEntry(rowIndex, columnIndex, number)
  }
}.flatMap(x => x)
val satellite_blockMatrix: BlockMatrix = new CoordinateMatrix(sat_byColumnAndRow).toBlockMatrix()

[Stage 15:====================================================>   (35 + 2) / 37]

satellite_cells_sizeB = Broadcast(20)
satellite_mat = org.apache.spark.mllib.linalg.distributed.RowMatrix@161d14e3
sat_byColumnAndRow = MapPartitionsRDD[64] at flatMap at <console>:68
satellite_blockMatrix = org.apache.spark.mllib.linalg.distributed.BlockMatrix@314b66b


org.apache.spark.mllib.linalg.distributed.BlockMatrix@314b66b

### SC

In [9]:
//SC
val sc_exists = fs.exists(new org.apache.hadoop.fs.Path(sc_path))
var Sc :BlockMatrix = null
if (sc_exists) {
  val rdd_indexed_rows :RDD[IndexedRow]= sc.objectFile(sc_path)
  Sc = new IndexedRowMatrix(rdd_indexed_rows).toBlockMatrix()
} else {
  val satellite_M_1_Gc = sc.parallelize(Array[Vector](satellite_summary.mean)).map(m => Vectors.dense(m.toArray))
  val satellite_M_1_Gc_RowM: RowMatrix = new RowMatrix(satellite_M_1_Gc)
  val sat_M_1_Gc_byColumnAndRow = satellite_M_1_Gc_RowM.rows.zipWithIndex.map {
    case (row, rowIndex) => row.toArray.zipWithIndex.map {
      case (number, columnIndex) => new MatrixEntry(rowIndex, columnIndex, number)
    }
  }.flatMap(x => x)
  val satellite_M_1_Gc_blockMatrix = new CoordinateMatrix(sat_M_1_Gc_byColumnAndRow).toBlockMatrix()

  val sat_matrix_Nt_1 = new Array[Double](satellite_grids.count().toInt)
  //satellite_grids.unpersist(false)
  for (i <- 0 until sat_matrix_Nt_1.length)
    sat_matrix_Nt_1(i) = 1
  val satellite_M_Nt_1 = sc.parallelize(sat_matrix_Nt_1).map(m => Vectors.dense(m))
  val satellite_M_Nt_1_RowM: RowMatrix = new RowMatrix(satellite_M_Nt_1)
  val sat_M_Nt_1_byColumnAndRow = satellite_M_Nt_1_RowM.rows.zipWithIndex.map {
    case (row, rowIndex) => row.toArray.zipWithIndex.map {
      case (number, columnIndex) => new MatrixEntry(rowIndex, columnIndex, number)
    }
  }.flatMap(x => x)
  val satellite_M_Nt_1_blockMatrix = new CoordinateMatrix(sat_M_Nt_1_byColumnAndRow).toBlockMatrix()
  val satellite_M_Nt_Gc_blockMatrix = satellite_M_Nt_1_blockMatrix.multiply(satellite_M_1_Gc_blockMatrix)

  //Sc = satellite_blockMatrix.subtract(satellite_M_Nt_Gc_blockMatrix)
  val joined_mat :RDD[ (Long, (Array[Double], Array[Double]))] = satellite_blockMatrix.toIndexedRowMatrix().rows.map(m => (m.index, m.vector.toArray)).join(satellite_M_Nt_Gc_blockMatrix.toCoordinateMatrix().toIndexedRowMatrix().rows.map(m => (m.index, m.vector.toArray)))
  Sc = (new CoordinateMatrix(joined_mat.map {case (row_index, (a,b)) => a.zip(b).map(m => m._1-m._2).zipWithIndex.map{ case (v,col_index) => new MatrixEntry(row_index, col_index,v)}}.flatMap(m => m))).toBlockMatrix() 

  //save to disk
  Sc.toIndexedRowMatrix().rows.saveAsObjectFile(sc_path)
}
//Sc.persist(StorageLevel.DISK_ONLY)

sc_exists = true
Sc = org.apache.spark.mllib.linalg.distributed.BlockMatrix@7e7eeab4


org.apache.spark.mllib.linalg.distributed.BlockMatrix@7e7eeab4

### Model Data

In [10]:
//Model
val model_cells_sizeB = sc.broadcast(model_cells_size)
val model_mat: RowMatrix = new RowMatrix(model_grids.map(m => m.zipWithIndex).map(m => m.filter(!_._1.isNaN)).map(m => Vectors.sparse(model_cells_sizeB.value.toInt, m.map(v => v._2), m.map(v => v._1))))

// Split the matrix into one number per line.
val mod_byColumnAndRow = model_mat.rows.zipWithIndex.map {
  case (row, rowIndex) => row.toArray.zipWithIndex.map {
    case (number, columnIndex) => new MatrixEntry(rowIndex, columnIndex, number)
  }
}.flatMap(x => x)
val model_blockMatrix: BlockMatrix = new CoordinateMatrix(mod_byColumnAndRow).transpose().toBlockMatrix()

[Stage 18:====================================================>   (34 + 2) / 36]

model_cells_sizeB = Broadcast(26)
model_mat = org.apache.spark.mllib.linalg.distributed.RowMatrix@7cb62cb3
mod_byColumnAndRow = MapPartitionsRDD[81] at flatMap at <console>:68
model_blockMatrix = org.apache.spark.mllib.linalg.distributed.BlockMatrix@4c4e703f


org.apache.spark.mllib.linalg.distributed.BlockMatrix@4c4e703f

### Mc

In [11]:
//MC
val mc_exists = fs.exists(new org.apache.hadoop.fs.Path(mc_path))
var Mc :BlockMatrix = null
if (mc_exists) {
  val rdd_indexed_rows :RDD[IndexedRow]= sc.objectFile(mc_path)
  Mc = new IndexedRowMatrix(rdd_indexed_rows).toBlockMatrix()
} else {
  val model_M_1_Gc = sc.parallelize(Array[Vector](model_summary.mean)).map(m => Vectors.dense(m.toArray))
  val model_M_1_Gc_RowM: RowMatrix = new RowMatrix(model_M_1_Gc)
  val mod_M_1_Gc_byColumnAndRow = model_M_1_Gc_RowM.rows.zipWithIndex.map {
    case (row, rowIndex) => row.toArray.zipWithIndex.map {
      case (number, columnIndex) => new MatrixEntry(rowIndex, columnIndex, number)
    }
  }.flatMap(x => x)
  val model_M_1_Gc_blockMatrix = new CoordinateMatrix(mod_M_1_Gc_byColumnAndRow).toBlockMatrix()

  val model_matrix_Nt_1 = new Array[Double](model_grids.count().toInt)
  //model_grids.unpersist(false)

  for (i <- 0 until model_matrix_Nt_1.length)
    model_matrix_Nt_1(i) = 1
  val model_M_Nt_1 = sc.parallelize(model_matrix_Nt_1).map(m => Vectors.dense(m))
  val model_M_Nt_1_RowM: RowMatrix = new RowMatrix(model_M_Nt_1)
  val mod_M_Nt_1_byColumnAndRow = model_M_Nt_1_RowM.rows.zipWithIndex.map {
    case (row, rowIndex) => row.toArray.zipWithIndex.map {
      case (number, columnIndex) => new MatrixEntry(rowIndex, columnIndex, number)
    }
  }.flatMap(x => x)
  val model_M_Nt_1_blockMatrix = new CoordinateMatrix(mod_M_Nt_1_byColumnAndRow).toBlockMatrix()
  val model_M_Nt_Gc_blockMatrix = model_M_Nt_1_blockMatrix.multiply(model_M_1_Gc_blockMatrix)
  val model_M_Gc_Nt_blockMatrix = model_M_Nt_Gc_blockMatrix.transpose
  
  //Mc = model_blockMatrix.subtract(model_M_Gc_Nt_blockMatrix)
  val joined_mat :RDD[ (Long, (Array[Double], Array[Double]))] = model_blockMatrix.toIndexedRowMatrix().rows.map( m => (m.index, m.vector.toArray)).join(model_M_Gc_Nt_blockMatrix.toIndexedRowMatrix().rows.map(m => (m.index, m.vector.toArray)))
  Mc = (new CoordinateMatrix(joined_mat.map {case (row_index, (a,b)) => a.zip(b).map(m => m._1-m._2).zipWithIndex.map{ case (v,col_index) => new MatrixEntry(row_index, col_index,v)}}.flatMap(m => m))).toBlockMatrix()

  //save to disk
  Mc.toIndexedRowMatrix().rows.saveAsObjectFile(mc_path)
}
//Mc.persist(StorageLevel.DISK_ONLY)

[Stage 20:=======================================>           (766 + 234) / 1000]

mc_exists = true
Mc = org.apache.spark.mllib.linalg.distributed.BlockMatrix@1e0d3bf3


org.apache.spark.mllib.linalg.distributed.BlockMatrix@1e0d3bf3

### Sr

In [12]:
//SR
var Sr :BlockMatrix = null

Sr: org.apache.spark.mllib.linalg.distributed.BlockMatrix = null


### Mr

In [13]:
//MR
var Mr :BlockMatrix = null

Mr: org.apache.spark.mllib.linalg.distributed.BlockMatrix = null


## Matrix Multiplication

In [14]:
//Matrix Multiplication
var matrix_mul :IndexedRowMatrix = null
var n_components :Long = 0

//Normal Matrix
if (matrix_mode == 0) {
  matrix_mul = model_blockMatrix.toIndexedRowMatrix().multiply(satellite_blockMatrix.toLocalMatrix())
  n_components = List(model_blockMatrix.numRows(), model_blockMatrix.numCols(), satellite_blockMatrix.numRows(), satellite_blockMatrix.numCols()).min
}

//SC Matrix
if (matrix_mode == 1) {
  matrix_mul = Mc.toIndexedRowMatrix().multiply(Sc.toLocalMatrix())
  n_components = List(Mc.numRows(), Mc.numCols(), Sc.numRows(), Sc.numCols()).min
}

//SR Matrix
if (matrix_mode == 2) {
  matrix_mul = Mr.toIndexedRowMatrix().multiply(Sr.toLocalMatrix())
  n_components = List(Mr.numRows(), Mr.numCols(), Sr.numRows(), Sr.numCols()).min
}

[Stage 25:=================================================>        (6 + 1) / 7]

matrix_mul = org.apache.spark.mllib.linalg.distributed.IndexedRowMatrix@46ac09ef
n_components = 26


26

## SVD

In [15]:
val svd: SingularValueDecomposition[RowMatrix, Matrix] = matrix_mul.toRowMatrix().computeSVD(n_components.toInt, computeU = true)

[Stage 234:============================================>          (25 + 6) / 31]

svd = 


SingularValueDecomposition(org.apache.spark.mllib.linalg.distributed.RowMatrix@45e8b92c,[1.2051734846237378E11,1.5731502063346007E8,9.354714854448463E7,6.905679842999004E7,5.526621808388971E7,4.0701929023372E7,3.212782160191781E7,2.4903252813016314E7,1.9320312318715412E7,1.82663003730079E7,1.7190574025181644E7,1.5019956259419689E7,1.2725605528641628E7,1.1646307097874615E7,1.0860080572826205E7,1.043934269453451E7,9931441.01827396,8331478.349791396,8252190.491014826,7354727.077607888,6655639.861438513,5983963.356948634,5924921.769039156,5790605.120873877,5170085.782857946,4426607.045517704],-0.0020369236563774515  -1.2352680576094898E-4 ...


SingularValueDecomposition(org.apache.spark.mllib.linalg.distributed.RowMatrix@45e8b92c,[1.2051734846237378E11,1.5731502063346007E8,9.354714854448463E7,6.905679842999004E7,5.526621808388971E7,4.0701929023372E7,3.212782160191781E7,2.4903252813016314E7,1.9320312318715412E7,1.82663003730079E7,1.7190574025181644E7,1.5019956259419689E7,1.2725605528641628E7,1.1646307097874615E7,1.0860080572826205E7,1.043934269453451E7,9931441.01827396,8331478.349791396,8252190.491014826,7354727.077607888,6655639.861438513,5983963.356948634,5924921.769039156,5790605.120873877,5170085.782857946,4426607.045517704],-0.0020369236563774515  -1.2352680576094898E-4  ... (26 total)
-0.0020369236563774536  -1.2352680576104455E-4  ...
-0.002050085953087252   -9.656678126447997E-5   ...
-0.0020482374188388014  -6.239900059127528E-5   ...
-0.0020469477474922906  -8.282905757388564E-5   ...
-0.002046921195694613   -8.469442990960812E-5   ...
-0.0020494828957270854  -1.1646198380386294E-4  ...
-0.0020482163771796263  -1.07

In [16]:
val U: RowMatrix = svd.U // The U factor is a RowMatrix.
val s: Vector = svd.s // The singular values are stored in a local dense vector.
val V: Matrix = svd.V // The V factor is a local dense matrix.
val S = Matrices.diag(s)

U = org.apache.spark.mllib.linalg.distributed.RowMatrix@45e8b92c
s = [1.2051734846237378E11,1.5731502063346007E8,9.354714854448463E7,6.905679842999004E7,5.526621808388971E7,4.0701929023372E7,3.212782160191781E7,2.4903252813016314E7,1.9320312318715412E7,1.82663003730079E7,1.7190574025181644E7,1.5019956259419689E7,1.2725605528641628E7,1.1646307097874615E7,1.0860080572826205E7,1.043934269453451E7,9931441.01827396,8331478.349791396,8252190.491014826,7354727.077607888,6655639.861438513,5983963.356948634,5924921.769039156,5790605.120873877,5170085.782857946,4426607.045517704]
V = 


-0.0020369236563774536  ...


-0.0020369236563774515  -1.2352680576094898E-4  ... (26 total)
-0.0020369236563774536  -1.2352680576104455E-4  ...
-0.002050085953087252   -9.656678126447997E-5   ...
-0.0020482374188388014  -6.239900059127528E-5   ...
-0.0020469477474922906  -8.282905757388564E-5   ...
-0.002046921195694613   -8.469442990960812E-5   ...
-0.0020494828957270854  -1.1646198380386294E-4  ...
-0.0020482163771796263  -1.0750978256375236E-4  ...
-0.002048860666861444   -9.424934725535012E-5   ...
-0.0020456599060407563  -9.284860894097065E-5   ...
-0.002045659906040756   -9.284860894073978E-5   ...
-0.0020362532465157206  -1.415676743759653E-4   ...
-0.0020356343955318067  -1.7376906063041402E-4  ...
-0.0020469166743874332  -9.285184924842558E-5   ...
-0.0020475679661214967  -8.566994728675451E-5   ...
-0.00204505826853663    -4.8193003072109784E-5  ...
-0.002043100741237972   -2.23306785942502E-5    ...
-0.002046888408163146   -1.0471313699443129E-4  ...
-0.00204753281119614    -1.2259768761988018E-4  ...
-

# Save results

### R

In [17]:
U.rows.map(m => m.toArray.mkString(",")).repartition(1).saveAsTextFile(out_path + "U_Sc.csv")

[Stage 237:===================================================>(998 + 2) / 1000]

### V

In [23]:
sc.parallelize(V.rowIter.toVector.map(m => m.toArray.mkString(",")),4).saveAsTextFile(out_path + "V_Sc.csv")

[Stage 241:==========================================>              (3 + 1) / 4]

lastException: Throwable = null


### S

In [24]:
sc.parallelize(S.rowIter.toVector.map(m => m.toArray.mkString(",")),1).saveAsTextFile(out_path + "S_Sc.csv")